One hot encoding vector  


In [13]:
import pandas as pd
df = pd.read_csv('./data/class2.csv')

label = df['class2']

label.unique() # ['N', 'I', 'B']

from sklearn import preprocessing
onehot_encoder = preprocessing.OneHotEncoder()
onehot_encoder.fit(label.values.reshape(-1, 1))

onehot_vector = onehot_encoder.transform(label.values.reshape(-1, 1)).toarray()
onehot_vector

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

TF-IDF  
단어 $w$가 문서 $d$ 내에서 얼마나 중요한가  
tf : 현재 문서에서 등장하는 단어의 빈도수  
idf : 이 단어가 나오는 문서의 총 갯수   

BoW에서 문서 내 단어 빈도수를 고려하되, 다른 문서에도 너무 자주 나오는 건(ex-a, the) 중요도를 낮추어 점수를 계산합니다   
가벼운 NLP 문제에서는 BoW, tf-idf라는 간단한 모델을 먼저 써보시는 것을 추천합니다. ‌  

단점   
순서가 중요한 문제에는 쓰기 힘들다  
vocabulary가 커지면 커질수록 쓰기 힘들다.  

https://jiho-ml.com/weekly-nlp-2/  

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc = ["I like machine learning", 'I love deep learning', 'I run everyday']
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = tfidf_matrix * tfidf_matrix.T    

doc_distance.toarray()

array([[1.      , 0.224325, 0.      ],
       [0.224325, 1.      , 0.      ],
       [0.      , 0.      , 1.      ]])

word2vec은 예측 기반 임베딩으로 딥러닝 기반 자연어 처리에서는 통계 기반의 카운터 벡터나 tf-idf보다 더 자주 사용됨.  

In [32]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings("ignore")
import gensim
from gensim.models import Word2Vec


In [33]:
sample = open("./data/peter.txt", "r", encoding="utf-8")
s = sample.read()

f = s.replace("\n", " ") # 줄 바꿈을 공백으로 변환
data = []

for i in sent_tokenize(f): # 문장 세그멘테이션
    temp = []
    for j in word_tokenize(i): # 단어 세그멘테이션
        temp.append(j.lower()) # 소문자화
    data.append(temp)

data # 일단 peter.txt의 내용을 단어로 다 쪼갬 (영어라서 그럼, 한국어면 형태소 별로 쪼갤 것)

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['after',
  'wendy',
  'had',
  'tucked',
  'her',
  'younger',
  'brothers',
  'jhon',
  'and',
  'michael',
  'to',
  'bed',
  ',',
  'she',
  'went',
  'to',
  'read',
  'a',
  'book',
  '.'],
 ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'],
 ['he', 'was', 'flying', '.'],
 ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'],
 ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'],
 ['“', 'hello', '!'],
 ['who', 'are', 'you', '?'],
 ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'],
 ['“', 'my', 'name', 'is', 'peter', 'pan', '.'],
 ['my',
  'shadow',
  'wouldn',
  '’',
  't',
  'stock',
  'to',
  'me.',
  '”',
  ',',
  'he',
  'rep

word2vec: CBOW(continuous bag of words) 방식

In [36]:
# 위에서 만든 data를 이용해서 단어 벡터를 만들어보자
model1 = Word2Vec(
    data, # 단어 
    min_count=1, # 최소 빈도수 제한
    vector_size=100, # 임베딩된 벡터의 차원
    window=5, # window 크기
    sg=0, # 0일 때는 CBOW, 1일 때는 skip-gram (default는 CBOW)
    )
model1.wv.similarity('peter', 'wendy') # peter와 wendy의 유사도
model1.wv.most_similar('eat', topn=3) # peter와 가장 유사도가 높은 단어 3개 가져와 봐

[('peter', 0.34764593839645386),
 ('read', 0.2823920249938965),
 ('had', 0.27761027216911316)]

word2vec: skip-gram 방식  
CBOW보다 성능도 좋고 더 널리 사용되므로 일반적으로 word2vec하면 skip-gram 방식의 word2vec을 말함   
그런데 실제 사용해보니 뭐가 더 좋다고 할 수 없을 정도로 좀 결과가 다른데...?  
뭐가 더 좋다고 결론 내리기보다 좀 더 생각을 해보고 쓸 것  

In [38]:
model2 = Word2Vec(
    data, # 단어 
    min_count=1, # 최소 빈도수 제한
    vector_size=100, # 임베딩된 벡터의 차원
    window=5, # window 크기
    sg=1, # 0일 때는 CBOW, 1일 때는 skip-gram (default는 CBOW)
    )
model2.wv.similarity('peter', 'wendy') # peter와 wendy의 유사도
model1.wv.most_similar('eat', topn=3) # peter와 가장 유사도가 높은 단어 3개 가져와 봐

[('peter', 0.34764593839645386),
 ('read', 0.2823920249938965),
 ('had', 0.27761027216911316)]

fasttext(enhanced skip-gram)  


word2vec의 개량 버전으로 단어를 n-gram 기반 subword로 나누고, skip-gram을 통해서 각 subword에 대한 embedding vector에 주변 단어의 context vector를 곱하여 더한다. 그리고 이 값이 최대가 되도록 학습을 수행한다.  

각 subword의 embedding vector의 합이 word embedding vector가 된다.  

In [41]:
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText(
    "./data/peter.txt", 
    vector_size=4, # 임베딩의 크기
    window=3, # 3 gram 기반 subword 쪼갬
    min_count=1,
    epochs=10 
)

model.wv.similarity('peter', 'wendy') # peter와 wendy의 유사도

0.45924556

성능을 위해서 pre train된 word vector를 활용해보자  
https://fasttext.cc/docs/en/pretrained-vectors.html  

In [42]:
from gensim.models import KeyedVectors
model_kr = KeyedVectors.load_word2vec_format("./data/wiki.ko.vec")

In [43]:
word = "공부"

for i in model_kr.similar_by_word(word): # 공부와 가장 유사한 단어 
    print(i)

('공부만', 0.7568907737731934)
('공부가', 0.7530936002731323)
('공부나', 0.7502861618995667)
('공부함', 0.7442659139633179)
('공부와', 0.7337692379951477)
('공부법', 0.7335165739059448)
('공부는', 0.7331703305244446)
('공부다', 0.7323489189147949)
('공부방', 0.7313159704208374)
('공부로', 0.7305828928947449)
